In [28]:
import openai
import pinecone
import langchain
import os
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from PyPDF2 import PdfReader
import tiktoken
import cassio
from dotenv import load_dotenv


In [33]:
load_dotenv(override=True)

True

In [36]:
ASTRA_DB_APPLICATION_TOKEN = os.getenv('ASTRA_DB_APPLICATION_TOKEN')
ASTR_DB_ID = os.getenv('ASTR_DB_ID')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [13]:
pdfreader = PdfReader("India Tourism Statistics 2019.pdf")

In [14]:
from typing_extensions import Concatenate
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text+=content

In [15]:
#this is all the text present in the pdf
raw_text

"india tourism statistics, 2019\nGovernment of india\nministry of t ourism\nmarket r esearch divisionoFFicErs oF marKEt rEsEarcH diVision \nassociatEd WitH tHE PrEP aration oF india \ntourism statistics, 2019\nshri. P.c. cyriac\nadditional director General\nsh. Fakhre alam\ndirector\nms. a qsa ilahi dy. director\nms. Kirti Gaikwad dy. director\nshri s.K. mohanta Programmer\nsmt. charu arora dEo Grade. “c”contEnts\nchapter Page\nchapter- 1 important s tatistics on t ourism 1\nchapter-2 inbound t ourism 7\n2.1 inbound t ourism in india 7\nTable 2.1.1 Foreign Tourist Arrivals (FTAs), Arrivals of Non-Resident Indians (NRI) \nand International Tourist Arrivals (ITAs) 1981-20187-8\nTable 2.1.2 Month-wise break-up of Non Residents Indians (NRIs) Arrivals, 2016-20188\nTable 2.1.3 Month-wise break-up of International Tourist Arrivals (ITAs) in India \n2016-20189\nTable 2.1.4 Nationality wise Foreign Tourist Arrivals in India 2016-2018 10-13\n2.2 Foreign t ourist arrivals in india from different

Initialize ASTRADB

In [32]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTR_DB_ID)

Create an embedding object from langchain

In [21]:
llm = OpenAI(openai_api_key = OPENAI_API_KEY)
embedding =  OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)


c:\Users\HP\Documents\GitHub\PDF_Query_LLM_Project\llmenv_pdf_query\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAI instead.
  warn_deprecated(
c:\Users\HP\Documents\GitHub\PDF_Query_LLM_Project\llmenv_pdf_query\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


Create Langchain Vector Store----AStraDB

In [22]:
astra_vector_store = Cassandra(
    embedding = embedding, # whenevr we push data into cassadra astradb, it will convert all text into vector using this embedding
    table_name= 'pdfquery01',
    session = None,
    keyspace = None
)

Converting data into text chunks

In [23]:
text_splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len
)
texts = text_splitter.split_text(raw_text)

Created a chunk of size 1070, which is longer than the specified 800
Created a chunk of size 826, which is longer than the specified 800
Created a chunk of size 877, which is longer than the specified 800
Created a chunk of size 825, which is longer than the specified 800
Created a chunk of size 811, which is longer than the specified 800
Created a chunk of size 853, which is longer than the specified 800
Created a chunk of size 964, which is longer than the specified 800
Created a chunk of size 883, which is longer than the specified 800
Created a chunk of size 843, which is longer than the specified 800
Created a chunk of size 891, which is longer than the specified 800
Created a chunk of size 824, which is longer than the specified 800


In [24]:
len(texts)

423

In [25]:
astra_vector_store.add_texts(texts) # it will be adding the chuncked texts in astraDB in the embedded form.
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [26]:
query_text = 'Which month in 2018 in india had highest number of nri tourists and how many?'
print('\n Question: \'%s\'' %query_text )
answer = astra_vector_index.query(query_text, llm=llm).strip()
print('\n Answer: \'%s\'' %answer)


 Question: 'Which month in 2018 in india had highest number of nri tourists and how many?'

 Answer: 'The month of February in 2018 had the highest number of NRI tourists at 453,298 arrivals.'
